In [10]:
import pandas as pd
import mysql.connector
from datetime import datetime
import numpy as np

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('Sample-Data-Historic.csv')

# Convert the date column to the correct format
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')

# Handle missing values by replacing NaN with None
df.replace({np.nan: None}, inplace=True)

# Define column data types based on the sample data
column_data_types = {
    'ticker': 'VARCHAR(255)',
    'date': 'DATE',
    'revenue': 'BIGINT',
    'gp': 'BIGINT',
    'fcf': 'BIGINT',
    'capex': 'BIGINT'
}

table_creation_query = """
CREATE TABLE IF NOT EXISTS stocks (
    ticker VARCHAR(255),
    date DATE,
    revenue BIGINT,
    gp BIGINT,
    fcf BIGINT,
    capex BIGINT,
    PRIMARY KEY (ticker, date),
    INDEX ticker_date_idx (ticker, date)
)
"""

# Connect to MySQL
conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="password",
    database="assignment_backend"
)
cursor = conn.cursor()

# Execute the table creation query
cursor.execute(table_creation_query)

# Insert data into the table
for _, row in df.iterrows():
    insert_query = "INSERT INTO stocks (ticker, date, revenue, gp, fcf, capex) VALUES (%s, %s, %s, %s, %s, %s)"
    cursor.execute(insert_query, tuple(row))

# Commit the transaction and close the connection
conn.commit()
conn.close()